# `nn.ipynb`
This file aims to use a neural network to solve the multiple-peaks problem. There is information in the peaks and positions of the nodes, but we haven't been able to draw it out yet. Maybe a neural network can.

## NN specs
* Input
    * Positions of N nodes
    * Peaks of N-1 MIs, the peak closest to zero
    * Period: avg period of each signal using MI between signal and self
    * Total: 3N inputs
* Output
    * N-1 peaks/time delays

More details below...

Liberties (assumptions) taken:
* When finding the period, you need an upper bound. This could be deduced using the speed and the width of the space, but I am just using a fraction of the signal length. I happen to know that one fifth of the length of the signal (`2000 // 5 = 400`, or `1600 // 6 = 320`) always contains the period. 

Other notes:
* This is the first time I am implementing a randomization of the target location in FN data, of a sort. There will be a bounding box with a minimum size of 50 in each direction, where all the target nodes have to live inside. This way, we do not need to regenerate the FN data each time, but the target is still at a random position relative to the nodes. 

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from scipy.signal import find_peaks

from matplotlib import pyplot as plt

from tqdm import tqdm

from WSN import *
import mutual_information as mtin

In [2]:
mtin.default_fn_equ_params["dt"] = 0.01
mtin.default_fn_equ_params["T"] = 20_000
mtin.default_fn_equ_params['stim'] = [[[250, 350], [45, 65], [45, 65]]]
start_frame = int(40 / mtin.default_fn_equ_params["dt"])
mtin.default_fn_equ = FHN(**mtin.default_fn_equ_params)

In [3]:
if mtin.default_fn_sol is None:
    mtin.default_fn_sol = np.load("default_fn_sol_dt_0.01.npy")

In [4]:
# mtin.solve_default_fn_equ()

In [5]:
# np.save("default_fn_sol_dt_0.01.npy", mtin.default_fn_sol)

In [30]:
# Generate signals and find all peaks
def get_fn_peaks_inside_period(self: WSN, r0=None, use_mst=False):
    if r0 is None:
        r0 = np.array([55, 55])
    results = self.transmit_continuous(signal_type="fn", start_frame=start_frame)
    assert np.all(np.array([result[0] for result in results]) == self.nodes)

    self.printv("Finding period")
    avg_period = 0
    for i, sigi in results:
        # This is arbitrary, but I know this will work for this case
        max_tau = len(sigi) // 5
        shifts = np.arange(-max_tau, max_tau, 1)
        mis = mi_shift(np.transpose([sigi, sigi]), shifts)

        mis = np.array(mis)
        max_peak = mis.max()
        peaks, _ = find_peaks(mis)
        peaks = peaks[np.argsort(mis[peaks])]
        peaks = peaks[-5:]
        peaks.sort()
        period1 = peaks[1] - peaks[0]
        period2 = peaks[2] - peaks[1]
        period = (period1 + period2) / 2
        dt = mtin.default_fn_equ_params["dt"]
        period *= dt
        avg_period += period
    avg_period /= len(results)
    self.printv("Period found to be", avg_period)

    all_peaks = []   # [(rn, rm, p, p0), ...]
    pair_to_ind = {} # {(n, m): i}

    # For now, tree must be a list so that the neural network
    # sees the peaks in the same order every time
    if use_mst:
        tree = list(self.find_MST())
    else:
        tree = [
            (None, 0, i)
            for i in range(1, len(self.nodes))
        ]
    for _, i, j in tree:
        ri, sigi = results[i]
        rj, sigj = results[j]
        dt = mtin.default_fn_equ_params["dt"]
        max_tau = int((2 * avg_period) / dt)
        shifts = np.arange(-max_tau, max_tau, 1)
        mis = mtin.mi_shift(np.transpose([sigi, sigj]), shifts)

        mis = np.array(mis)
        max_peak = mis.max()
        inclusion_factor = 0.5
        peaks, _ = find_peaks(mis, height=max_peak * inclusion_factor)
        if self.verbose and np.random.uniform(0, 1) < 0.1:
            plt.plot(shifts, mis)
            plt.show(block=False)
        if len(peaks) == 0:
            plt.plot(shifts * mtin.default_fn_equ_params["dt"], mis)
            plt.show(block=False)
            p = (dist(ri, r0) - dist(rj, r0)) / self.c
            raise ValueError(
                f"No peaks found between nodes {i} and {j} "
                f"at positions {ri} and {rj}.\n"
                f"The peak should be at time {p}."
            )
        peaks = shifts[peaks]
        # p0 = peak closest to 0
        p0 = peaks[np.argmin(np.abs(peaks))]

        # p = ideal peak
        p = (dist(ri, r0) - dist(rj, r0)) / self.c
        pair_to_ind[(i, j)] = len(all_peaks)
        all_peaks.append((ri, rj, p, p0))
    return all_peaks, pair_to_ind, avg_period


In [31]:
[setattr(WSN, attr, globals()[attr]) for attr in (
    "get_fn_peaks_inside_period",
)]

[None]

Create testing and training data

Input format:
* `x0, y0, x1, y1, ...`   (normalized by `wsn.size`)
* `p0, p1, p2, ...`       (normalized by period)
* `period`

Output format:
* `p0, p1, p2, ...`       (normalized by period)

In [8]:
N = 12
c = 1.6424885622140555
wsn = WSN(100, N, D=142, std=0, c=c, verbose=False)
wsn.reset_anchors(range(N))

In [9]:
def get_input_and_output(wsn: WSN):
    all_peaks, pair_to_ind, period = wsn.get_fn_peaks_inside_period()
    # Input format:
    # [
    #   wsn.nodes.flatten() -- len = 2N,
    #   peaks -- len = N-1,
    #   period -- len = 1
    # ]

    # Output format:
    # peaks -- len = N-1
    input = []
    output = []
    for ri, rj, p, p0 in all_peaks:
        input.append(p0 / period)
        output.append(p / period)
    input.append(period)
    input = np.concatenate((wsn.nodes.flatten() / 100, input))
    output = np.array(output)

    return input, output


In [13]:
data_size = 100
# input_shape = (data_size, 3 * N)
# output_shape = (data_size, N - 1)
input = [0] * data_size
output = [0] * data_size
for j in tqdm(range(data_size)):
    error = True
    while error:
        error = False
        try:
            wsn.reset_nodes()
            i, o = get_input_and_output(wsn)
            input[j] = i
            output[j] = o
        except ValueError as e:
            print(f"ValueError: {e}")
            error = True
input = np.array(input)
output = np.array(output)


  0%|          | 0/100 [00:00<?, ?it/s]d:\Repositories\WSN-Localization\mutual_information.py:105: RuntimeWarning: divide by zero encountered in log
  mi = -0.5 * np.log(np.linalg.det(cor))
100%|██████████| 100/100 [46:51<00:00, 28.12s/it]


In [14]:
np.save("nn_saves/input.npy", input)
np.save("nn_saves/output.npy", output)

In [15]:
num_train_samples = data_size * 8 // 10
train_input = input[:num_train_samples]
train_output = output[:num_train_samples]
test_input = input[num_train_samples:]
test_output = output[num_train_samples:]

Create model...

In [16]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(3*N,)),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(N-1, activation="linear")
])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mean_squared_error"])

In [17]:
model.fit(train_input, train_output, epochs=5)

Epoch 1/5


ValueError: in user code:

    File "d:\Repositories\WSN-Localization\vwsn\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "d:\Repositories\WSN-Localization\vwsn\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\Repositories\WSN-Localization\vwsn\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "d:\Repositories\WSN-Localization\vwsn\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "d:\Repositories\WSN-Localization\vwsn\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "d:\Repositories\WSN-Localization\vwsn\lib\site-packages\keras\engine\input_spec.py", line 264, in a

In [ ]:
test_loss, test_mse = model.evaluate(test_input, test_output)